In [3]:
import Pkg; Pkg.add("Tables")
using CSV, Tables
using DataFrames
using Statistics

Pkg.add("HTTP")
using HTTP
Pkg.add("JSON3")
using JSON3

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [13]:
classes_url = "https://fireroad.mit.edu/courses/all?full=true"

"https://fireroad.mit.edu/courses/all?full=true"

In [ ]:
function get_all_classes()
    """
    Returns an array of dictionaries for each class that is valid, meaning it has a rating,
    in/out-of-class hours, and is public (ensuring that each class also has subject_id, title, and semester).
    """
    response = HTTP.get(classes_url)
    if response.status == 200
        all_classes_info_raw = JSON3.read(String(response.body))  # List of dictionaries
    else
        println("Failed to fetch all_classes_data: ", response.status)
        return []
    end

    # Identify "bad" classes missing required fields or not public
    bad_data = Set([
        class_info["subject_id"] for class_info in all_classes_info_raw
        if !haskey(class_info, "rating") ||
           !haskey(class_info, "in_class_hours") ||
           !haskey(class_info, "out_of_class_hours") ||
           class_info["public"] == false
    ])

    # Filter out bad classes
    all_classes_info = [
        class_info for class_info in all_classes_info_raw
        if !(class_info["subject_id"] in bad_data)
    ]

    return all_classes_info
end

get_all_classes (generic function with 1 method)

In [15]:
all_classes = get_all_classes()
println("Number of valid classes: ", length(all_classes))  # should be 4703

Number of valid classes: 4703


In [18]:
om_response = HTTP.get("https://fireroad.mit.edu/courses/lookup/6.c571")
if om_response.status == 200
    opt_meth_info = JSON3.read(String(om_response.body))  # List of dictionaries
end 
opt_meth_info

JSON3.Object{Base.CodeUnits{UInt8, String}, Vector{UInt64}} with 25 entries:
  :has_final           => true
  :description         => "Introduction to the methods and applications of opti…
  :offered_fall        => true
  :offered_spring      => false
  :meets_with_subjects => ["6.C57", "15.C57", "IDS.C57"]
  :instructors         => ["A. Jacquillat, H. Lu"]
  :joint_subjects      => ["15.C571"]
  :total_units         => 12
  :related_subjects    => ["6.C57", "IDS.C57", "6.7220", "6.7250", "6.C06", "15…
  :pdf_option          => false
  :is_half_class       => false
  :schedule            => "Lecture,E62-233/MW/0/1-2.30,E62-262/MW/0/4-5.30,E62-…
  :level               => "U"
  :prerequisites       => "18.C06/''permission of instructor''"
  :subject_id          => "6.C571"
  :title               => "Optimization Methods"
  :lab_units           => 0
  :design_units        => 0
  :public              => true
  :offered_summer      => false
  :lecture_units       => 4
  :preparation_units  

In [8]:
all_classes[1]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 28 entries:
  :rating              => 5.4
  :gir_attribute       => "REST"
  :has_final           => false
  :description         => "Presents engineering problems in a computational set…
  :offered_fall        => false
  :offered_spring      => true
  :meets_with_subjects => ["1.001"]
  :instructors         => ["J. Williams"]
  :out_of_class_hours  => 7.99
  :total_units         => 12
  :related_subjects    => ["2.156", "1.205", "1.C51", "1.000", "1.631", "1.063"…
  :pdf_option          => false
  :in_class_hours      => 5.7
  :is_half_class       => false
  :level               => "U"
  :prerequisites       => "GIR:CAL1"
  :subject_id          => "1.00"
  :title               => "Engineering Computation and Data Science"
  :lab_units           => 2
  :design_units        => 0
  :public              => true
  :offered_summer      => false
  :lecture_units       => 3
  :

In [16]:
R = [class[:rating] for class in all_classes];
H_raw = [class[:in_class_hours] + class[:out_of_class_hours] for class in all_classes];
H = round.(H_raw, digits=2)
U = [class[:total_units] for class in all_classes];

n = length(all_classes);
S = 8;

### run the following lines to get the data as csv files

In [17]:
# uncomment and run each of the lines below to save the files

# mkdir("data")

open("data/all_classes_info.json", "w") do file 
    JSON3.write(file, all_classes)
end

# CSV.write("data/ratings.csv",  Tables.table(R), writeheader=false)
# CSV.write("data/hours.csv",  Tables.table(H), writeheader=false)
# CSV.write("data/units.csv",  Tables.table(U), writeheader=false)
# CSV.write("data/variables.csv", Tables.table([n, S]), writeheader=false)

5733134